In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

#establish all team abbreviations to allow paramaterization of url, note some team abv are old names as that is the url basketball-reference uses
#ex: NJN (New Jersey Nets) for Brooklyn Nets
teams = ['ATL','BOS','NJN','CHA','CHI','CLE','DAL','DEN','DET','GSW','HOU','IND',
         'LAC','LAL','MEM','MIA','MIL','MIN','NOH','NYK','OKC','ORL','PHI',
         'PHO','POR','SAC','SAS','TOR','UTA','WAS']

#stats of interest, g (games), fg (field goals), trb (total rebounds), ast (assist), stl (steal), blk (block)
df = pd.DataFrame(index=teams, columns=["Most Games","Most Field Goals","Most Rebounds","Most Assists","Most Steals","Most Blocks"])
aID = ["leaders_g","leaders_fg","leaders_trb","leaders_ast","leaders_stl","leaders_blk"]

for i in range(len(teams)):
    #pull HTML for each franchise
    url = "https://www.basketball-reference.com/teams/"+teams[i]+"/leaders_career.html"

    page = requests.get(url)

    soup = BeautifulSoup(page.content,'html.parser')

    franchiseRow = []
    for j in range(len(aID)):
        #for each franchise find each of the accomplishments of interest
        temp = soup.find("div",{"id": aID[j]})
        player = temp.find('td',attrs={'class':'who'}).text.replace('*','')
        #stat for future versions if needed
        #stat = temp.find('td',attrs={'class':'value'}).text

        franchiseRow.append(player)
    #replace null values with acquired values
    df.loc[teams[i]] = franchiseRow

#replace outdated team names with current franchise name, (NJN,NOP)
df.rename(index={'NJN':'BKN'},inplace=True)
df.rename(index={'NOH':'NOP'},inplace=True)

#convert to CSV
df.to_csv('franchiseGreats.csv')